In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [3]:
bc = nx.betweenness_centrality(g)

In [4]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
})

In [5]:
char_infos = pd.read_csv('../dataset/characters_stats.csv', index_col=0)
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [6]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [7]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')
semi_df = semi_df[semi_df['Alignment'] != 'neutral']
semi_df = semi_df.replace(['good', 'bad'], [0, 1])

In [8]:
semi_df['Intelligence (%)'] = semi_df['Intelligence']/semi_df['Total']
semi_df['Power (%)'] = semi_df['Power']/semi_df['Total']
semi_df['Strength (%)'] = semi_df['Strength']/semi_df['Total']
semi_df['Combat (%)'] = semi_df['Combat']/semi_df['Total']

In [9]:
import statsmodels.api as sm

# model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Strength (%)', 
#                                                        'Power', 'Power (%)', 'Alignment']])
model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power', 'Alignment']])
result = model.fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.233
Model:                            OLS   Adj. R-squared (uncentered):              0.217
Method:                 Least Squares   F-statistic:                              14.00
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                    5.44e-10
Time:                        10:43:36   Log-Likelihood:                          636.14
No. Observations:                 188   AIC:                                     -1264.
Df Residuals:                     184   BIC:                                     -1251.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence     0.0001   2.72e-05      3.695      0.000    4.68e-05       0.000
Strength      1.605e-05   2.23e-05      0.719      0.473    -2.8e-05    6.01e-05
Power        -3.193e-05   2.81e-05     -1.134      0.258   -8.75e-05    2.36e-05
Alignment       -0.0034      0.001     -2.448      0.015      -0.006      -0.001
==============================================================================
Omnibus:                      225.161   Durbin-Watson:                   0.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8120.973
Skew:                           4.930   Prob(JB):                         0.00
Kurtosis:                      33.651   Cond. No.                         210.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
print(f"Coef:\n{result.params}\n")
print(f"std err:\n{result.bse}\n")
print(f"p-values:\n{result.pvalues}\n")

Coef:
Intelligence    0.000100
Strength        0.000016
Power          -0.000032
Alignment      -0.003350
dtype: float64

std err:
Intelligence    0.000027
Strength        0.000022
Power           0.000028
Alignment       0.001368
dtype: float64

p-values:
Intelligence    0.000290
Strength        0.473044
Power           0.258083
Alignment       0.015295
dtype: float64



In [11]:
# g = fm.load('marvelWeights.gml')
# g.label_nodes('name')
# g.set_all_nodes(size=15, labpos='hover')
# g.set_all_edges(color=(0, 0, 0, 0.5))

# intelligence = {}

# for i in g.nodes:
#     intelligence[i] = float(g.nodes[i]['intelligence'])

# g.scale_nodes_size(intelligence)
# g.draw()